## ETL de Carga de información de OFEI1204.txt

## Installation of base libraries

In [1]:
import pandas as pd
from unidecode import unidecode
import re
from config.db_config import engine

## Extraction

In [2]:
def lectura_archivo(archivo):
    datos = []
    agente_actual = None

    try:
        with open(archivo, "r", encoding="UTF-8") as archivo:
            for linea in archivo:
                linea = linea.strip()
                if linea.startswith("AGENTE:"):
                    agente_actual = linea.split("AGENTE:")[1].strip()
                elif (
                    linea
                    and not linea.startswith("Ofertas Iniciales")
                    and agente_actual is not None
                ):
                    partes = linea.split(",")
                    tipo = partes[1].strip()
                    planta = partes[0].strip()
                    valores = partes[2:]
                    datos.append([agente_actual, planta, tipo] + valores)
    except Exception as e:
        print("Error al cargar el archivo:", e)

    return datos

In [3]:
# Uso de la función
datos = lectura_archivo("../data/OFEI1204.txt")
num_columnas_valores = len(datos[0]) - 3
nombres_columnas_valores = [f"Hora_{i+1}" for i in range(num_columnas_valores)]

## Transformation

In [4]:
# Creación del DataFrame
df = pd.DataFrame(
    datos, columns=["Agente", "Planta", "Tipo"] + nombres_columnas_valores
)
# Filtro los datos según el tipo 'D'
df["Tipo"] = df["Tipo"].str.upper()
df_tipo_d = df[df["Tipo"] == "D"]
df_tipo_d = df_tipo_d.copy()

In [5]:
# Aplicar unidecode y luego reemplazar guiones y paréntesis por espacios
df_tipo_d["Agente"] = df_tipo_d["Agente"].apply(lambda x: unidecode(x).replace('-', ' ').replace('(', ' ').replace(')', ' '))
df_tipo_d["Planta"] = df_tipo_d["Planta"].apply(lambda x: unidecode(x).replace('-', '').replace('(', ' ').replace(')', ' '))

# Limpieza de los campos Agente, Planta
df_tipo_d["Agente"] = df_tipo_d["Agente"].apply(lambda x: re.sub(r'[^0-9a-zA-Z\s]', "", x))
df_tipo_d["Planta"] = df_tipo_d["Planta"].apply(lambda x: re.sub(r'[^0-9a-zA-Z\s]', "", x))

In [6]:
df_tipo_d["Id"] = range(1, len(df_tipo_d) + 1)

In [7]:
# Definición de columnas de datos finales
df_final = df_tipo_d[
    ["Id", "Agente", "Planta"] + df_tipo_d.columns[3:-1].tolist()
].copy()

In [8]:
# Garantizar el tipo de dato de los campos
convertir_float = df_tipo_d.columns[3:-1].tolist()
df_final[convertir_float] = df_final[convertir_float].astype("float64")

df_final[["Agente", "Planta"]] = df_final[["Agente", "Planta"]].astype(str)
df_final = df_final.copy()

## Load

In [9]:
def carga_df_tabla(dataframe, engine, table_name):
    try:
        conn = engine.connect()
        columns = dataframe.columns.tolist()
        dataframe.to_sql(name=table_name, con=engine, if_exists="replace", index=False)
        print(f"Los datos se han cargado correctamente en la tabla {table_name}")
    except Exception as e:
        print(f"Error al cargar datos desde el DataFrame a la tabla {table_name}: {e}")
    finally:
        conn.close()

In [10]:
carga_df_tabla(df_final, engine, "tbl_ofei1204")

Los datos se han cargado correctamente en la tabla tbl_ofei1204
